In [ ]:
!pip install optuna

In [1]:
import transformers
print(transformers.__version__)
import torch
print(torch.__version__)
!python --version


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


TensorFlow Version: 2.13.0
CUDA Version TensorFlow was built with: 11.8
cuDNN Version TensorFlow was built with: 8
PyTorch Version: 2.0.0
CUDA Version PyTorch was built with: 11.8
cuDNN Version: 8900
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


0

In [ ]:
!pip install ViennaRNA

In [ ]:
!pip install ViennaRNA pandas

In [ ]:
!pip install tqdm
!pip install seaborn
!pip install plotly


In [ ]:
import pandas as pd
import numpy as np
import json
import tensorflow.keras.layers as L
from keras.layers import Dropout
import seaborn as sns
import tensorflow as tf
import plotly.express as px
from sklearn.preprocessing import quantile_transform,StandardScaler,MinMaxScaler
from transformers import BertConfig,TFBertModel,BertModel
from keras.callbacks import EarlyStopping
import random
import math, json, os, random
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from kerastuner.tuners import Hyperband
from kerastuner.tuners import RandomSearch
from kerastuner import HyperParameters
import kerastuner as kt
from sklearn import ensemble
from sklearn import datasets
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import KFold

np.random.seed(100)
random.seed(100)

train = pd.read_json('/kaggle/input/stanford-covid-vaccine/train.json', lines=True)
test = pd.read_json('/kaggle/input/stanford-covid-vaccine/test.json', lines=True)
sample_df = pd.read_csv('/kaggle/input/stanford-covid-vaccine/sample_submission.csv')


#sanity check
train.head()


AUTO = tf.data.experimental.AUTOTUNE
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)
# This will tell us the columns we are predicting
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']


sorted(train['signal_to_noise'].apply(np.round).astype(int).unique())
np.bincount(train['signal_to_noise'].apply(np.round).astype(int))
sorted(train['SN_filter'].apply(np.round).astype(int).unique()) 
np.bincount(train['SN_filter'].apply(np.round).astype(int))
train = train[train['signal_to_noise'] > .25]

In [ ]:
train.info()

**DATA AUGMENTATION (NCBI + OPPENVACCINE)**

In [ ]:
# import torch
# import numpy as np

# # Assuming `generated_sequences` is loaded from 'generated_sequences2.npy' and contains softmax probabilities
# generated_sequences = np.load('/kaggle/input/gan-sequences/gen_sequences.npy')

    
# import RNA
# import pandas as pd
# from tqdm import tqdm

# import RNA
# import numpy as np
# import pandas as pd

# # Load your NumPy array containing RNA sequences
# sequences_array = np.load('/kaggle/input/gan-sequences/gen_sequences.npy')

# # Define a function to predict RNA structure for a given sequence
# def predict_rna_structure(sequence):
#     ss, _ = RNA.fold(sequence)
#     return ss

# # Open a file to write the results incrementally
# with open('GAN_RNAseqstr.csv', 'w') as file:
#     file.write("sequence,structure\n")  # Write the header

#     # Iterate over each sequence in the NumPy array
#     for seq in sequences_array:
#         # Extract the sequence as a string
#         sequence_str = seq[0]
        
#         # Predict the RNA structure for the sequence
#         structure = predict_rna_structure(sequence_str)
        
#         # Write the sequence and its structure to the CSV file
#         file.write(f"\"{sequence_str}\",\"{structure}\"\n")

# # Read and print the dataframe
# df = pd.read_csv('GAN_RNAseqstr.csv')
# print(df)


In [ ]:
# def read_file_in_chunks(file_path, chunk_size=1024):
#     with open(file_path, 'r') as file:
#         while True:
#             chunk = file.read(chunk_size)
#             if not chunk:
#                 break
#             yield chunk


# # Specify the file paths
# file_paths = [
#     '/kaggle/input/sequences-ncbi-auto/downloaded_sequences_0.fasta',
# #     '/kaggle/input/sequences-ncbi-auto/downloaded_sequences_1.fasta',
# #     '/kaggle/input/sequences-ncbi-auto/downloaded_sequences_2.fasta',
# #     '/kaggle/input/sequences-ncbi-auto/downloaded_sequences_3.fasta',
# #     '/kaggle/input/sequences-ncbi-auto/downloaded_sequences_4.fasta',
# #     '/kaggle/input/sequences-ncbi-auto/downloaded_sequences_5.fasta'
# #     '/kaggle/input/sequences-ncbi-auto/downloaded_sequences_6.fasta'
# ]

# contents = [''.join(read_file_in_chunks(file_path)) for file_path in file_paths]#3463170954
# contents_without_mRNA = [line for line in contents[0].split('\n') if not line.startswith('>')]

# updated_contents = ''.join(contents_without_mRNA)


    
# import RNA
# import pandas as pd
# from tqdm import tqdm

# def predict_rna_structure(sequence):
#     ss, _ = RNA.fold(sequence)
#     return ss

# def chunk_sequence(sequence, chunk_size=107):
#     for i in range(0, len(sequence), chunk_size):
#         yield sequence[i:i + chunk_size]

# def calculate_total_chunks(sequence, chunk_size=107):
#     return (len(sequence) + chunk_size - 1) // chunk_size

# chunked_sequences = chunk_sequence(updated_contents)
# total_chunks = calculate_total_chunks(updated_contents)

# # Open a file to write the results incrementally (For NCBI GAN generated sequences)
# with open('RNAseqstr.csv', 'w') as file:
#     file.write("sequence,structure\n")  # Write the header

#     for seq in tqdm(chunked_sequences, total=total_chunks, desc="Processing Sequences"):
#         structure = predict_rna_structure(seq)
#         file.write(f"\"{seq}\",\"{structure}\"\n")

# # Read and print the dataframe
# df = pd.read_csv('RNAseqstr.csv')
# print(df)

# # Open a file to write the results incrementally (For NCBI RNA SEQUENCES)
# with open('RNAseqstr.csv', 'w') as file:
#     file.write("sequence,structure\n")  # Write the header

#     for seq in tqdm(chunked_sequences, total=total_chunks, desc="Processing Sequences"):
#         structure = predict_rna_structure(seq)
#         file.write(f"\"{seq}\",\"{structure}\"\n")

# # Read and print the dataframe
# df = pd.read_csv('RNAseqstr.csv')
# print(df)

In [ ]:
NCBI_with_struc = pd.read_csv('/kaggle/input/rna-seq-str/RNAseqstr.csv')
def convert_t_to_u(sequence):
    return sequence.replace('T', 'U')

# Apply this function to every entry in the 'sequence' column
NCBI_with_struc['sequence'] = NCBI_with_struc['sequence'].apply(convert_t_to_u)

# file_path = '/kaggle/input/stanford-covid-vaccine/train.json'
# data = pd.read_json(file_path, lines=True)
# type(data)

# print(type(data),data.shape,data.columns)

# print(type(NCBI_with_struc),NCBI_with_struc.shape,NCBI_with_struc.columns)


In [ ]:
# import pandas as pd

# # Assuming data and data2 are your DataFrames

# # Merge the DataFrames
# merged_df = pd.merge(data, NCBI_with_struc, on=['sequence', 'structure'], how='outer')

# # Replace NaNs with space in the columns that are exclusive to 'data'
# exclusive_columns = data.columns.difference(NCBI_with_struc.columns)
# merged_df[exclusive_columns] = merged_df[exclusive_columns].fillna(' ')

# # Now merged_df is your final DataFrame


In [ ]:
# merged_df
GAN_RNAseqstr = pd.read_csv('/kaggle/input/rna-seq-str/GAN_RNAseqstr.csv')
print(len(NCBI_with_struc))

NCBI_with_struc = pd.merge(GAN_RNAseqstr, NCBI_with_struc, on=['sequence', 'structure'], how='outer')
len(NCBI_with_struc)

**MASKING: (structure, sequence)**

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertConfig
from torch.optim.lr_scheduler import OneCycleLR
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForMaskedLM
from torch.optim import AdamW
import numpy as np
from transformers import BertConfig
import matplotlib.pyplot as plt
train_losses = []
val_losses = []

# file_path = '/kaggle/input/stanford-covid-vaccine/train.json'
# data = pd.read_json(file_path, lines=True)  ####merged_df
data = NCBI_with_struc
# Define the masking probability
masking_probability = 0.25
token2int = {'A': 1, 'C': 2, 'G': 3, 'U': 4, '(': 5, ')': 6, '.': 7, '[MASK]': 8}
# Custom Dataset for RNA
class RNADataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        masked_sequence = self.data.iloc[idx]['masked_sequence']
        masked_structure = self.data.iloc[idx]['masked_structure']
        original_sequence = self.data.iloc[idx]['original_sequence']
        return masked_sequence, masked_structure, original_sequence

def collate_fn(batch):
    sequences, structures, original_sequences = zip(*batch)
    sequence_ids = []
    structure_ids = []
    label_ids = []

    # Determine the maximum length in this batch
    max_length = max(max(len(seq) for seq in sequences), max(len(struct) for struct in structures))

    for seq, struct, orig_seq in zip(sequences, structures, original_sequences):
        seq_ids = [token2int.get(token, token2int['[MASK]']) for token in seq]
        struct_ids = [token2int.get(token, token2int['[MASK]']) for token in struct]
        orig_seq_ids = [token2int.get(token, token2int['[MASK]']) for token in orig_seq]

        # Padding
        seq_ids.extend([token2int['[MASK]']] * (max_length - len(seq_ids)))
        struct_ids.extend([token2int['[MASK]']] * (max_length - len(struct_ids)))
        orig_seq_ids.extend([token2int['[MASK]']] * (max_length - len(orig_seq_ids)))

        sequence_ids.append(seq_ids)
        structure_ids.append(struct_ids)
        label_ids.append(orig_seq_ids)

    sequence_ids = torch.tensor(sequence_ids)
    structure_ids = torch.tensor(structure_ids)
    label_ids = torch.tensor(label_ids)

    input_ids = torch.stack([sequence_ids, structure_ids], dim=1)
    attention_masks = torch.ones_like(sequence_ids)

    return {'input_ids': input_ids, 'attention_mask': attention_masks, 'labels': label_ids}

def mask_dynamic(entity, dynamic_type):
    if dynamic_type == 'contiguous':
        start = np.random.randint(0, len(entity)-1)
        end = np.random.randint(start, len(entity))
        entity = entity[:start] + '[MASK]' * (end - start + 1) + entity[end + 1:]
    elif dynamic_type == 'gapped':
        gaps = np.random.choice([True, False], len(entity), p=[0.2, 0.8])
        entity = ''.join([e if not g else '[MASK]' for e, g in zip(entity, gaps)])
    return entity

def mask_sequence(sequence):
    mask = np.random.rand(len(sequence)) < masking_probability
    masked_positions = [i for i, m in enumerate(mask) if m]  # List to store the positions where masking occurs
    masked_sequence = ''.join(['[MASK]' if m else seq for seq, m in zip(sequence, mask)])
    return masked_sequence, masked_positions

def mask_structure(structure, masked_positions,sequence):
    
    masked_structure = ''
    for i, char in enumerate(structure):
        if i in masked_positions:
            masked_structure += '[MASK]'  # Apply masking at the same positions as in the sequence
        if sequence[i] == 'G' and np.random.rand() < 0.3:  # 30% chance to mask if nucleotide is G
            masked_structure += '[MASK]'  # Apply masking at positions where sequence has 'G'
        else:
            masked_structure += char

    return masked_structure


# Assuming 'data' is your DataFrame containing the RNA sequences and structures
for idx, row in NCBI_with_struc.iterrows():
    masked_sequence, masked_positions = mask_sequence(row['sequence'])
    masked_structure = mask_structure(row['structure'], masked_positions,row['sequence'])
    NCBI_with_struc.at[idx, 'masked_sequence'] = masked_sequence
    NCBI_with_struc.at[idx, 'masked_structure'] = masked_structure

print(NCBI_with_struc[['masked_sequence', 'masked_structure']])


In [ ]:
len(NCBI_with_struc)

In [ ]:

class CustomBERTModel(nn.Module):
    def __init__(self, config):
        super(CustomBERTModel, self).__init__()
        self.sequence_embedding = nn.Embedding(config.vocab_size, config.hidden_size)
        self.structure_embedding = nn.Embedding(config.vocab_size, config.hidden_size)

        # Flattening layer: adjust the input size as needed
        self.flatten = nn.Linear(2 * config.hidden_size, config.hidden_size)

        # BertModel contains the transformer layers
        self.bert = BertModel(config)
        self.classifier = nn.Linear(config.hidden_size, config.vocab_size)  # Assuming the output size is vocab_size

    def forward(self, input_ids, attention_mask):
        # Check the shape of input_ids to confirm it's a 3D tensor
        if input_ids.dim() != 3 or input_ids.size(1) != 2:
            raise ValueError(f"Expected input_ids of shape [batch_size, 2, sequence_length], got {input_ids.shape}")

        seq_input_ids = input_ids[:, 0, :]  # Sequence component
        struct_input_ids = input_ids[:, 1, :]  # Structure component


        # Get embeddings
        seq_embeddings = self.sequence_embedding(seq_input_ids)
        struct_embeddings = self.structure_embedding(struct_input_ids)

        # Concatenate and flatten embeddings
        combined_embeddings = torch.cat((seq_embeddings, struct_embeddings), dim=2)
        flat_embeddings = self.flatten(combined_embeddings)

        # Get the last hidden state
        bert_output = self.bert(inputs_embeds=flat_embeddings, attention_mask=attention_mask)
        last_hidden_state = bert_output.last_hidden_state

        # Predictions
        predictions = self.classifier(last_hidden_state)

        return predictions
# Initialize the Dataset and DataLoader

NCBI_with_struc['original_sequence'] = NCBI_with_struc['sequence']
dataset = RNADataset(NCBI_with_struc)
print(type(dataset), dataset)


from sklearn.model_selection import train_test_split

# Assuming 'data' is your DataFrame containing the RNA sequences and structures
# Split data into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Create training and validation datasets
train_dataset = RNADataset(train_data)
val_dataset = RNADataset(val_data)

# Create DataLoaders for training and validation datasets
train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=16, collate_fn=collate_fn)

# data_loader = DataLoader(dataset, batch_size=8, collate_fn=collate_fn)

# Config for BERT
bert_config = BertConfig(
    vocab_size=800,  # Example vocab size
    hidden_size=128,
    num_hidden_layers=8,
    num_attention_heads=8,
    intermediate_size=500
)

# Initialize the model
model = CustomBERTModel(bert_config)

# Example input (batch_size=2, sequence_length=107)
input_ids = torch.randint(0, 800, (2, 2, 107))
attention_mask = torch.ones(2, 107)

# Forward pass
outputs = model(input_ids, attention_mask)


# Initialize BERT model from scratch
optimizer = AdamW(model.parameters(), lr=1e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training Loop
model.train()
num_epochs = 2
# Assuming you are using CrossEntropyLoss for a masked language modeling task
loss_function = nn.CrossEntropyLoss()

# Learning rate scheduler setup
scheduler = OneCycleLR(optimizer, max_lr=1e-4, steps_per_epoch=len(train_loader), epochs=num_epochs)

# Early stopping parameters
early_stopping_patience = 5
best_val_loss = float('inf')
epochs_without_improvement = 0

for epoch in range(num_epochs):
    epoch_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}")

    for batch in progress_bar:
        inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
        labels = batch['labels'].to(device)

        # Forward pass
        predictions = model(**inputs)  # Directly use the model's output as predictions

        # Reshape labels to match the output format and compute loss
        labels = labels.view(-1)  # Flatten labels if necessary
        predictions = predictions.view(-1, predictions.size(-1))  # Reshape for CrossEntropyLoss

        loss = loss_function(predictions, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()  # Update the learning rate

        # Accumulate loss
        epoch_loss += loss.item()
        progress_bar.set_postfix({'loss': epoch_loss / (progress_bar.last_print_n + 1)})

    train_losses.append(epoch_loss / len(train_loader))

    print(f'Epoch: {epoch + 1}, Loss: {epoch_loss / len(train_loader)}')



    # Validation phase
    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        for batch in val_loader:  # Assuming you have a validation dataloader
            inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
            labels = batch['labels'].to(device)

            predictions = model(**inputs)
            labels = labels.view(-1)
            predictions = predictions.view(-1, predictions.size(-1))
            loss = loss_function(predictions, labels)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    print(f'Validation Loss: {val_loss}')

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), f'model_epoch_{epoch+1}.pt')  # Save best model
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= early_stopping_patience:
            print("Early stopping triggered")
            break

        
model_save_path = 'bert_for_masked_lm_rna.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")
plt.figure(figsize=(10, 5))
plt.title("Training and Validation Loss")
plt.plot(train_losses, label="Training loss")
plt.plot(val_losses, label="Validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()


In [ ]:

token2int = {'A': 1, 'C': 2, 'G': 3, 'U': 4, '(': 5, ')': 6, '.': 7, 'B': 8, 'E': 9, 'H': 10, 'I': 11, 'M': 12, 'S': 13, 'X': 14, '[MASK]': 15}

# def preprocess_inputs(df, cols=['sequence', 'structure', 'predicted_loop_type']):
def preprocess_inputs(df, cols=['sequence']):

    base_fea = np.transpose(np.array(df[cols].applymap(lambda seq: [token2int[x] for x in seq]).values.tolist()),(0, 2, 1))
    print(base_fea.shape)
    return np.transpose(np.concatenate([base_fea], 2),(0, 2, 1))

train_inputs = preprocess_inputs(train)
train_labels = np.array(train[pred_cols].values.tolist()).transpose((0, 2, 1))
# train_inputs = torch.tensor(train_inputs, dtype=torch.float32)
# train_labels = torch.tensor(train_labels, dtype=torch.float32)

# trainLabels = train[pred_cols]
# print("trainLabels.shape",trainLabels.shape)

# public_df = test.query("seq_length == 107").copy()
# private_df = test.query("seq_length == 130").copy()

# public_inputs = preprocess_inputs(public_df)
# private_inputs = preprocess_inputs(private_df)


In [ ]:
# print(train_inputs[0].shape)
# print(train_labels[0].shape)

In [ ]:
# print(np.array_equal(train_inputs[0], train_inputs1[0][4]))
# print(train_labels1[0] == train_labels[0])  
# print(train_inputs1[0][0].shape)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
from sklearn.model_selection import KFold
from transformers import BertConfig, BertModel, BertTokenizer
import matplotlib.pyplot as plt
from transformers import BertModel, BertTokenizer


# Define constants
kf = KFold(n_splits=4, shuffle=True, random_state=42)
pred_len = 68
MAX_EPOCHS = 350
Seq_Len = 107
batch_size = 64
train_losses = []
val_losses = []

# Initialize lists for storing fold-wise average losses
fold_train_losses = []
fold_val_losses = []

# Initialize lists to store training histories
histories = []


# Define custom loss function
def mcrmse(y_true, y_pred):
    colwise_mse = torch.mean((y_true - y_pred) ** 2, dim=1)
    mcrmse = torch.mean(torch.sqrt(colwise_mse))
    return mcrmse

class RNADegradationModel(nn.Module):
    def __init__(self, custom_bert_model, seq_len=107, pred_len=68, hidden_dim=64):
        super(RNADegradationModel, self).__init__()
        self.bert = custom_bert_model.bert
        bert_hidden_size = self.bert.config.hidden_size  # Access hidden size from the BERT model
   
        self.truncate = nn.Linear(seq_len * bert_hidden_size, pred_len * hidden_dim)
        self.output = nn.Linear(hidden_dim, 5)  # Output 5 values per position
        self.hidden_dim = hidden_dim  # Ensure this is correctly initialized
       
    def forward(self, input_ids, attention_mask=None):
        bert_embeddings = self.bert(input_ids, attention_mask=attention_mask)[0]
        batch_size = bert_embeddings.size(0)
        flattened_embeddings = bert_embeddings.view(batch_size, -1)

        # Ensure the size matches with self.truncate input size
        truncated = self.truncate(flattened_embeddings)  # Adjusted to the correct shape
        truncated = truncated.view(-1, pred_len, self.hidden_dim)

        all_outputs = []
        for i in range(pred_len):
            prediction = truncated[:, i, :]
            output_i = self.output(prediction)
            all_outputs.append(output_i.unsqueeze(1))

        predicted_output = torch.cat(all_outputs, dim=1)
        return predicted_output


custom_bert_model = CustomBERTModel(bert_config)
custom_bert_model.load_state_dict(torch.load('bert_for_masked_lm_rna.pth'))

train_losses_per_fold = []  # List to store losses for all folds

for fold, (idxT, idxV) in enumerate(kf.split(train_inputs)):
    fold_specific_train_losses = []
    fold_specific_val_losses = []

    custom_model = RNADegradationModel(custom_bert_model, seq_len=107, pred_len=68, hidden_dim=bert_config.hidden_size)
    custom_model.to(device)

    # Convert NumPy arrays to PyTorch tensors
    trn_ = torch.tensor(train_inputs[idxT, :, :], dtype=torch.float32).to(device)
    trn_labs = torch.tensor(train_labels[idxT, :, :], dtype=torch.float32).to(device)

    val_ = torch.tensor(train_inputs[idxV, :, :], dtype=torch.float32).to(device)
    val_labs = torch.tensor(train_labels[idxV, :, :], dtype=torch.float32).to(device)



    optimizer = optim.AdamW(custom_model.parameters(), lr=1e-4)
#     optimizer = optim.SGD(custom_model.parameters(), lr=1e-4)
    train_inputs = torch.tensor(train_inputs, dtype=torch.float32).to(device)
    train_labels = torch.tensor(train_labels, dtype=torch.float32).to(device)

    accumulation_steps = 4  # Adjust as needed
    SAVE_INTERVAL = 5  # Save the model every 5 epochs

    for epoch in range(MAX_EPOCHS):
        custom_model.train()
        total_loss = 0
        num_batches = int(np.ceil(len(train_inputs) / batch_size))

        for i in range(num_batches):
            optimizer.zero_grad()  # Clear gradients at the start of each batch

            for j in range(accumulation_steps):
                # Calculate start and end indices for the current accumulated batch
                start_index = i * batch_size + j * (batch_size // accumulation_steps)
                end_index = start_index + (batch_size // accumulation_steps)

                # Ensure not to exceed the array bounds
                end_index = min(end_index, len(train_inputs))
                if end_index <= start_index:
                    continue  # Skip this iteration if the batch is empty

                # Fetch the sub-batch data
                inputs = train_inputs[start_index:end_index].squeeze(1)
                inputs = inputs.long().to(device)
                labels = train_labels[start_index:end_index]
                labels = labels.to(device)

                # Forward pass and loss computation
                outputs = custom_model(inputs)
                loss = mcrmse(labels, outputs)
                loss = loss / accumulation_steps  # Normalize the loss (important when accumulating)

                # Backward pass (accumulates gradients)
                loss.backward()
                total_loss += loss.item()

            # Perform the optimization step after accumulating gradients
            optimizer.step()

        avg_train_loss = total_loss / num_batches
        train_losses.append(avg_train_loss)

    #     torch.save(custom_model.state_dict(), f'model_epoch_{epoch+1}.pt')
        print(f"Epoch {epoch+1}/{MAX_EPOCHS}, Train Loss: {total_loss / num_batches}")

        num_val_batches = int(np.ceil(len(val_) / batch_size))

        # Validation
        custom_model.eval()
        total_val_loss = 0  # Initialize total validation loss
        with torch.no_grad():
            for i in range(num_val_batches):
                start_index = i * batch_size
                end_index = start_index + batch_size

                val_inputs = val_[start_index:end_index]
                val_labels = val_labs[start_index:end_index]

                # Ensure correct dimensions for val_inputs
                if val_inputs.dim() > 2:
                    val_inputs = torch.squeeze(val_inputs, dim=1)
                if val_inputs.dim() > 2:
                    val_inputs = val_inputs.view(-1, val_inputs.size(-1))
                val_inputs = val_inputs.long()

                val_outputs = custom_model(val_inputs)
                val_batch_loss = mcrmse(val_labels, val_outputs)
                val_loss = torch.mean(val_batch_loss)
                total_val_loss += val_loss.item()  # Accumulate the validation loss

        avg_val_loss = total_val_loss / num_val_batches  # Calculate the average validation loss
        val_losses.append(avg_val_loss)
        fold_specific_train_losses.append(avg_train_loss)
        fold_specific_val_losses.append(avg_val_loss)

        # Store the average validation loss for this epoch
        histories.append({'loss': total_loss / num_batches, 'val_loss': avg_val_loss})

        # Save the model's state_dict to a file

        if (epoch + 1) % 10 == 0 or (epoch + 1) == MAX_EPOCHS:
            torch.save(custom_model.state_dict(), f'model_epoch_{epoch+1}.pt')
            print(f"Saved model at fold {fold}, epoch {epoch + 1}")

            
    # Store losses
    fold_train_losses.append(fold_specific_train_losses)
    fold_val_losses.append(fold_specific_val_losses)


    # Plot training histories for each fold
    for fold, history in enumerate(histories):
        print(f"Fold {fold+1} - Training Loss:", history['loss'])
        print(f"Fold {fold+1} - Validation Loss:", history['val_loss'])


# # If you want to plot these as well, you can create lists and plot them
# fold_train_losses = [history['loss'] for history in histories]
# fold_val_losses = [history['val_loss'] for history in histories]


# Calculate average losses across folds
average_train_losses = np.mean(fold_train_losses, axis=0)
average_val_losses = np.mean(fold_val_losses, axis=0)

# Plot the average loss
plt.figure(figsize=(10, 5))
plt.title("Training and Validation Loss")
plt.plot(average_train_losses, label="Average Training loss")  # Plot average training loss
plt.plot(average_val_losses, label="Average Validation loss")  # Plot average validation loss
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# Inspect the first batch
first_batch = next(iter(train_loader))
print(type(first_batch))
print(len(first_batch))

# If it's a tuple or list, you can check the contents
if isinstance(first_batch, (tuple, list)):
    print([type(item) for item in first_batch])
    print([item.shape if isinstance(item, torch.Tensor) else None for item in first_batch])


In [ ]:
# import numpy as np
# import tensorflow as tf
# import time
# from sklearn.model_selection import KFold
# from transformers import BertConfig, TFBertModel
# from tensorflow.keras.layers import Input, Flatten, Dense
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# import matplotlib.pyplot as plt

# # Define constants
# kf = KFold(n_splits=4, shuffle=True, random_state=42)
# SEED = 1
# Seq_Len = 119
# Pred_Len = 68
# HYPERBAND_MAX_EPOCHS = 10
# MAX_TRIALS = 20
# EXECUTION_PER_TRIAL = 10
# N_EPOCH_SEARCH = 20

# Seq_Len = 107

# Define custom loss function
import torch

def mcrmse(y_true, y_pred):
    colwise_mse = torch.mean((y_true - y_pred) ** 2, dim=1)
    return torch.mean(torch.sqrt(colwise_mse), dim=1)

# # Initialize early stopping
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)

# # Function to build your model
# from transformers import BertTokenizer, TFBertModel
# import tensorflow as tf

# # Load the tokenizer associated with the pretrained model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Create an input layer for your sequences
# input_layer = tf.keras.layers.Input(shape=(Seq_Len,), dtype=tf.int32)

# # Load the pre-trained BERT model
# base_model = TFBertModel.from_pretrained('/kaggle/working/bert_for_masked_lm_rna')

# # Extract BERT embeddings for your input sequences
# bert_embeddings = base_model(input_layer)[0]

# # You can continue building your custom layers and outputs on top of the BERT embeddings
# # Example: Add a Dense layer for regression
# dense_layer = tf.keras.layers.Dense(units=5, activation='linear')(bert_embeddings)

# # Create the final model
# model = tf.keras.models.Model(inputs=input_layer, outputs=dense_layer)

# # Modify the build_model function to use your pretrained BERT model
# def build_model(seq_len=107, pred_len=68, dropout=0.5, embed_dim=100, hidden_dim=128):
#     ids = Input(shape=(5, seq_len,), dtype=tf.int32)

#     # Use your pretrained BERT model as the base model
#     bert_embeddings = base_model(ids)[0]

#     truncated = bert_embeddings[:, :pred_len, :]
#     out = Dense(5, activation='linear')(truncated)
#     model = tf.keras.Model(inputs=ids, outputs=out)

#     return model

# # The rest of your code remains the same


# # Record the start time
# start_time = time.time()

# # Initialize lists to store training histories
# histories = []

# # Loop through K-Folds
# for fold, (idxT, idxV) in enumerate(kf.split(train_inputs)):
#     trn_ = train_inputs[idxT, :, :]
#     trn_labs = train_labels[idxT, :, :]

#     val_ = train_inputs[idxV, :, :]
#     val_labs = train_labels[idxV, :, :]

#     # Build and train the model
#     model = build_model(seq_len=Seq_Len, pred_len=Pred_Len)
#     history = model.fit(
#         trn_, trn_labs,
#         validation_data=(val_, val_labs),
#         batch_size=64,
#         epochs=50,
#         callbacks=[
#             tf.keras.callbacks.ReduceLROnPlateau(),
#             ModelCheckpoint('model' + str(fold) + '.h5', save_weights_only=True, save_best_only=True)
#         ]
#     )
#     histories.append(history)

#     # Load model weights
#     model_short = build_model(seq_len=107, pred_len=107)
#     model_long = build_model(seq_len=130, pred_len=130)

#     model_short.load_weights('model' + str(fold) + '.h5')
#     model_long.load_weights('model' + str(fold) + '.h5')

#     if fold == 0:
#         public_preds = model_short.predict([public_inputs]) / 5
#         private_preds = model_long.predict([private_inputs]) / 5
#     else:
#         public_preds += model_short.predict([public_inputs]) / 5
#         private_preds += model_long.predict([private_inputs]) / 5

#     # Compute and print validation metrics
#     val_losses = []
#     y_preds_best = None
#     PRED_COLS = ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C', 'deg_pH10', 'deg_50C']
#     y_valid = val_labs
#     y_preds = model.predict(val_)

#     mcloss = mcrmse(val_labs, y_preds)
#     val_losses.append(mcloss)
#     print('np.min(val_losses)', np.min(val_losses), 'val_losses', np.mean(val_losses), 'mcloss', np.mean(mcloss))

#     if np.min(np.mean(val_losses)) == np.mean(mcloss):
#         y_preds_best = y_preds

#     print('y_preds_best shape, y_valid shape', y_valid.shape, y_preds_best.shape)
#     fig, ax = plt.subplots(1, 3, figsize=(24, 8))
#     for i, p in enumerate(PRED_COLS):
#         ax[2].scatter(y_valid[:, :, i].flatten(), y_preds_best[:, :, i].flatten(), alpha=0.5)

#     ax[2].legend(PRED_COLS)
#     ax[2].set_xlabel('y_true')
#     ax[2].set_ylabel('y_predicted')
#     plt.show()

# # Record the end time
# end_time = time.time()

# # Calculate the duration
# duration = end_time - start_time
# print("Duration:", duration)

# # Plot training histories
# results = {"models": ['bert', 'bert'], "histories": [histories, histories]}
# fig, ax = plt.subplots(1, len(results['histories']), figsize=(20, 10))
# fig2, ax2 = plt.subplots(1, len(results['histories']), figsize=(20, 10))

# for i, result in enumerate(results['histories']):
#     accum_loss, accum_val_loss, accum_mae, accum_mse = 0, 0, 0, 0
#     accum_tf_pearson, accum_val_tf_pearson = 0, 0

#     n = len(result)
#     for history in result:
#         accum_loss += np.array(history.history['loss'])
#         accum_val_loss += np.array(history.history['val_loss'])
#         accum_mae += np.array(history.history['mae'])
#         accum_mse += np.array(history.history['mse'])
#         accum_tf_pearson += np.array(history.history['tf_pearson'])
#         accum_val_tf_pearson += np.array(history.history['val_tf_pearson'])

#     avg_loss = accum_loss / n
#     avg_val_loss = accum_val_loss / n
#     avg_mae = accum_mae / n
#     avg_mse = accum_mse / n
